# 通过命令行的方式和系统交互
- os与commands模块
- subprocess模块
- subprocess.Popen模块
- 在早期的python版本中，主要通过os.system()，os.popen().read()等函数来执行命令行指令，还有很少使用的commands模块
- python2.4开始官方建议使用subprocess模块

# OS与commands模块
- os.system(command)：返回命令执行状态码，而将命令执行结果输出到屏幕
- os.popen(command).read()：可以获取命令执行结果，但是无法获取命令执行状态码
- commands.getstatusoutput(command)：返回一个元组(命令执行状态码，命令执行结果)
    - commands模块只存在于python2.7中，且不支持window平台

In [5]:
import os
retcode = os.system('dir')
print(retcode)

0


In [4]:
import os
ret = os.popen('dir').read()
print(ret)

 驱动器 F 中的卷是 use
 卷的序列号是 2ED3-1761

 F:\hyw\学习\python\python_jupyter\基础\实用函数 的目录

2018/11/19  08:57    <DIR>          .
2018/11/19  08:57    <DIR>          ..
2018/11/19  08:29    <DIR>          .ipynb_checkpoints
2018/11/19  08:29               749 encode和decode.ipynb
2018/11/19  08:29             1,685 list相关.ipynb
2018/11/19  08:29             3,672 sort和sorted.ipynb
2018/11/19  08:29             3,304 冒泡排序.ipynb
2018/11/19  08:29               587 字典相关.ipynb
2018/11/19  08:57             1,664 执行cmd命令 - os和subprocess.ipynb
2018/11/19  08:29             4,533 获取当前运行函数名称和类方法名称.ipynb
2018/11/19  08:29             1,357 随机生成中文.ipynb
               8 个文件         17,551 字节
               3 个目录 31,444,574,208 可用字节



# subprocess模块
- 常用函数：
    - subprocess.run()：3.5新增执行指定的命令，等待命令执行完成后返回一个包含执行结果的completedProcess类的实例
    - subprocess.call()：执行指定命令，返回命令执行状态，功能类似于os.system(cmd)
    - subprocess.check_call()：2.5中新增的函数，执行指定的命令，如果执行成功则返回状态码，否则抛出异常，功能等价于subprocess.run(...,check=True)
    - subprocess.check_output()：2.7中新增的函数，执行指定的命令，如果执行状态码为0则返回命令执行结果，否则抛出异常
    - subprocess.getoutput(cmd)：接收字符串格式的命令，执行命令并返回执行结果，其功能类似于os.popen(cmd).read()和commands.getoutput(cmd)
    - subprocess.getstatusoutput(cmd)：执行cmd命令，返回一个元组（命令执行状态，命令执行结果输出），其功能类似于commands.getstatusoutput()
- 3.5之后的版本，官方推荐使用subprocess.run()函数替代其他函数来使用subprocess模块的功能
- subprocess.run,call,check_call和check_outpu都是通过对subprocess.Popen的封装来实现的高级函数
- subporcess.getoutput，subprocess.getstatusoutput函数在3.3.4开始才支持windows平台

In [ ]:
import subprocess
subprocess.run(args,*,stdin=None,input=None,stdout=None,stderr=None,shell=False,timeout=None,check=False,universal_newlines=False)

subprocess.call(arges,*,stdin=None,stdout=None,stderr=None,shell=False,timeout=None)

subprocess.check_call(args,*,stdin=None,stdout=None,stderr=None,shell=False,timeout=None)

subprocess.check_output(args,*,stdin=None,stderr=None,shell=False,universal_newlines=False,timeout=None)

subprocess.getstatusoutput(cmd)

subprocess.getoutput(cmd)

'''
参数说明：
args：要执行的shell命令，默认应该是一个字符串序列,如['df','-Th']，也可以是一个字符串，如'df -Th',为字符串时需要把shell参数设置为True
shell：如果shell为True，那么指定的命令将通过shell执行
check：如果check参数的值为True，且执行命令的进程以非0状态码退出，则会抛出CalledProcessError的异常，且该异常对象会包含参数、退出状态码、以及stdout和stderr
stdout,stderr：
    - run函数默认不会捕获命令执行结果的正常输出和错误输出，如果向获取这些内容需要传递subprocess.PIPE,然后可以通过返回的CompletedProcess类实例的
    stdout,stderr属性或捕获对应的内容
    - call和check_call函数返回的是命令执行的状态码，而不是CompletedProcess类实例
    - check_output函数默认会返回命令执行结果
    - input：该参数是传递给Popen.communicate,通常该参数的值必须是一个字节序列，如果universal_newlines=True,则其值应该是一个字符串
    - universal_newlines：该参数影响的是输入与输出的数据格式，比如它默认值为False，此时stdout和stderr的输出是字节序列，当该参数的值设置为True
        时，stdout和stderr的输出是字符串
'''

# subprocess.CompletedProcess类介绍
- subprocess.run函数返回一个subprocess.CompletedProcess类
- 3.5版本后才存在，表示一个已技术进程的状态信息
- 属性
    - args：用于加载该进程的参数，可能是一个列表或一个字符串
    - returncode：子进程的退出状态码，通常情况下，退出码为0则表示进程成功运行了，一个负值-N表示整个子进程被信号N终止了
    - stdout：从子进程捕获的stdout，通常是一个字节序列
        - 如果run函数被调用时指定universal_newlines=True，则该属性值是一个字符串
        - 如果run函数被调用时指定stderr=subprocess.STDOUT,那么stdout和stderr将被整合在该属性中，且stderr将为None
    - stderr：从子进程捕获的stderr，它的值与stdout一样，是一个字节序列或一个字符串，如果stderr没有被捕获的话，它的值就为None
    - check_returncode()：如果returncode是一个非0值，则该方法会抛出CalledProcessError异常

# subprocess.Popen介绍
- 该类用于在一个新的进程中执行一个子程序
- 上面介绍的函数都是基于subprocess.Popen类实现的
- subporcess模块底层的进程创建和管理是由Popen类来处理的

In [ ]:
# subprocess.Popen的构造函数
class subprocess.Popen(args, bufsize=-1, executable=None, stdin=None, stdout=None, stderr=None, 
    preexec_fn=None, close_fds=True, shell=False, cwd=None, env=None, universal_newlines=False,
    startup_info=None, creationflags=0, restore_signals=True, start_new_session=False, pass_fds=())

'''
参数介绍
    args： 要执行的shell命令，可以是字符串，也可以是命令各个参数组成的序列。
            当该参数的值是一个字符串时，该命令的解释过程是与平台相关的，因此通常建议将args参数作为一个序列传递。
    bufsize：指定缓存策略，0表示不缓冲，1表示行缓冲，其他大于1的数字表示缓冲区大小，负数 表示使用系统默认缓冲策略
    stdin, stdout, stderr： 分别表示程序标准输入、输出、错误句柄
    preexec_fn： 用于指定一个将在子进程运行之前被调用的可执行对象，只在Unix平台下有效。
    close_fds： 如果该参数的值为True，则除了0,1和2之外的所有文件描述符都将会在子进程执行之前被关闭
    shell： 该参数用于标识是否使用shell作为要执行的程序，如果shell值为True，则建议将args参数作为一个字符串传递而不要作为一个序列传递
    cwd： 如果该参数值不是None，则该函数将会在执行这个子进程之前改变当前工作目录
    env： 用于指定子进程的环境变量，如果env=None，那么子进程的环境变量将从父进程中继承。如果env!=None，它的值必须是一个映射对象
    universal_newlines： 如果该参数值为True，则该文件对象的stdin，stdout和stderr将会作为文本流被打开，否则他们将会被作为二进制流被打开
    startupinfo和creationflags： 这两个参数只在Windows下有效，它们将被传递给底层的CreateProcess()函数，用于设置子进程的一些属性，如主窗口的外观，进程优先级等
'''

## Popen类的实例可以调用的方法
- Popen.poll()：用于检查子进程（命令）是否已经执行结束，没有结束返回None,结束后返回状态码
- Popen.wait(timeout=None)：等待子进程结束，并返回状态码，如果在timeout指定的秒数之后进程还没有结束，将会抛出TimeoutExpired异常
- Popen.communicate(input=None,timeout=None)：该方法可用来与进程进行交互，比如发送输出到stdin，从stdout和stderr读取数据，直到到达文件末尾
- Popen.send_signal(signal)：发送指定的信号给这个子进程
- Popen.terminate()：停止该子进程
- Popen.kill()：杀死该子进程